# Javascript demonstration

In [ ]:
!pip install papaparse;
!pip install jquery;

In [ ]:
%%javascript
// Import papaparse library
require.config({
    paths: {
        PapaParse: 'https://cdn.jsdelivr.net/npm/papaparse@5.3.0',
        jquery: 'https://code.jquery.com/jquery-3.6.0.min',
    }
});

// Create a function to handle the CSV data
window.handleCSVData = function (csvData) {
    // Display the data in a table
    var tableHTML = '<table>';
    for (var i = 0; i < csvData.length; i++) {
        tableHTML += '<tr>';
        for (var j = 0; j < csvData[i].length; j++) {
            tableHTML += '<td>' + csvData[i][j] + '</td>';
        }
        tableHTML += '</tr>';
    }
    tableHTML += '</table>';

    $('#csvTable').html(tableHTML);
};

// Load the CSV file
window.loadCSVFile = function (file) {
    PapaParse.parse(file, {
        complete: function (results) {
            // Call the handleCSVData function to display the data
            window.handleCSVData(results.data);
        }
    });
};

In [ ]:
<div id="csvTable"></div>

In [ ]:
%%javascript
// Replace 'your_csv_file.csv' with the path to your actual CSV file
var fileURL = 'your_csv_file.csv';
window.loadCSVFile(fileURL);